In [2]:
!pip install opencv-python scikit-learn numpy tensorflow
!pip install psutil pillow

In [3]:
# # 🕒 Start timer
# import time
# start_time = time.time()

In [4]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

KeyboardInterrupt: 

In [71]:
import psutil
import time

def print_system_usage(phase):
    memory = psutil.virtual_memory()
    cpu = psutil.cpu_percent(interval=1)  # Waits 1 sec to measure CPU usage
    print(f"[{phase}] CPU Usage: {cpu:.2f}% | RAM Usage: {memory.percent:.2f}% ({memory.used / 1e9:.2f} GB used)")


In [72]:
# Load pretrained MobileNetV2 without classification head

feature_extractor = MobileNetV2(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)  # Default input shape for pre-trained weights
)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [73]:
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)        # MobileNetV2-specific preprocessing
    return img_array[0]  # Remove batch dimension to keep consistency

In [74]:
def extract_features_from_folder(folder_path):
    X = []
    y = []

    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp")  # Add more if needed

    for label_name in os.listdir(folder_path):
        label_folder = os.path.join(folder_path, label_name)
        if not os.path.isdir(label_folder):
            continue

        label = 0 if label_name.lower() == "healthy" else 1  # Binary: healthy = 0, others = 1

        for img_file in os.listdir(label_folder):
            if not img_file.lower().endswith(valid_extensions):
                continue  # Skip non-image files

            img_path = os.path.join(label_folder, img_file)
            try:
                img = image.load_img(img_path, target_size=(224, 224))  # match input size
                img_array = image.img_to_array(img)
                img_array = preprocess_input(img_array)
                X.append(img_array)
                y.append(label)
            except Exception as e:
                print(f"❌ Skipped {img_path}: {e}")

    if not X:
        raise ValueError(f"No images found in {folder_path}")

    X = np.array(X)
    features = feature_extractor.predict(X, verbose=0, batch_size=1)
    return features, np.array(y)


In [75]:
# Set paths
train_path = "Potato/Train"
test_path = "Potato/Test"
val_path = "Potato/Val"

# Extract features
print("Extracting features...")
X_train, y_train = extract_features_from_folder(train_path)
X_test, y_test = extract_features_from_folder(test_path)
X_val, y_val = extract_features_from_folder(val_path)
print_system_usage("After feature extraction (Train)")

Extracting features...
[After feature extraction (Train)] CPU Usage: 24.20% | RAM Usage: 56.60% (9.58 GB used)


In [76]:
# Train logistic regression on extracted features
print("Training logistic regression...")
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)
print_system_usage("After training Logistic Regression")

Training logistic regression...
[After training Logistic Regression] CPU Usage: 23.60% | RAM Usage: 59.00% (9.97 GB used)


In [ ]:
# 🕒 Start timer
import time
start_time = time.time()

In [77]:
# Evaluate on test set
print("\n🎯 Test Set Results:")
test_preds = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, test_preds))
print_system_usage("After prediction (Test)")
print(classification_report(y_test, test_preds, target_names=["Healthy", "Unhealthy"]))


🎯 Test Set Results:
Accuracy: 1.0
[After prediction (Test)] CPU Usage: 13.30% | RAM Usage: 59.10% (10.00 GB used)
              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00        46
   Unhealthy       1.00      1.00      1.00        98

    accuracy                           1.00       144
   macro avg       1.00      1.00      1.00       144
weighted avg       1.00      1.00      1.00       144



In [ ]:
# 🕒 Total Time
end_time = time.time()
print(f"\n⏱️ Total Time Taken: {end_time - start_time:.2f} seconds")

In [78]:
# Confusion matrix
cm = confusion_matrix(y_test, test_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Healthy", "Unhealthy"])

In [1]:
# Optionally evaluate on validation set
print("\n📊 Validation Set Results:")
val_preds = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, val_preds))
print_system_usage("After prediction (Validation)")
print(classification_report(y_val, val_preds, target_names=["Healthy", "Unhealthy"]))


📊 Validation Set Results:


NameError: name 'model' is not defined

In [80]:
# # 🕒 Total Time
# end_time = time.time()
# print(f"\n⏱️ Total Time Taken: {end_time - start_time:.2f} seconds")


⏱️ Total Time Taken: 239.04 seconds
